## Model 1
In this model in our two-model layer, we start with input the recent historical stats of the teams in a game and predict the stats for the team in their pending matchup. 

In [120]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import Ridge
pd.options.mode.chained_assignment = None


In [2]:
games_df = pd.read_csv("clean_games_with_opp.csv", index_col=0)
games_df.reset_index(drop=True, inplace=True)
games_df.head()

,school,school_id,points,fumblesRecovered,rushingTDs,passingTDs,kickReturnYards,kickReturnTDs,kickReturns,kickingPoints,...,opposing_rushingTDs,opposing_points,side,thirdDownConverts,thirdDownAttempts,fourthDownConverts,fourthDownAttempts,completions,passAttempts,passCompletionPercentage
0,Navy,2426,17,0.0,2.0,0.0,64.0,0.0,4.0,5.0,...,1.0,34,A,4.0,12.0,1.0,1.0,2.0,4.0,0.500000
1,UAB,5,48,1.0,4.0,2.0,64.0,0.0,2.0,12.0,...,1.0,10,A,9.0,14.0,0.0,0.0,13.0,20.0,0.650000
2,UMass,113,7,0.0,0.0,1.0,95.0,0.0,4.0,1.0,...,2.0,30,A,3.0,11.0,1.0,1.0,9.0,22.0,0.409091
3,UCF,2116,24,1.0,2.0,1.0,142.0,0.0,5.0,6.0,...,1.0,26,A,5.0,13.0,2.0,2.0,12.0,22.0,0.545455
4,Houston,248,7,1.0,1.0,0.0,89.0,0.0,6.0,1.0,...,3.0,27,A,4.0,16.0,4.0,4.0,25.0,50.0,0.500000


In [3]:
# Dropping all columns with more than 25 nulls, then dropping the individual rows that have nulls
x = games_df.isna().sum().to_dict()
column_drop = []

for k, v in x.items():
    if v > 25 :
        column_drop.append(k)

# Additionally, these columns are weird formats or already covered by other variables (ie the possession stuff)
extend_list = ['completionAttempts','totalPenaltiesYards','possessionTime','possession_minutes','possession_seconds','year','week','school','points','opposing_points']

#column_drop.extend(extend_list)
column_drop = column_drop + extend_list

games_df.drop(columns=column_drop, inplace = True)
games_df.dropna(axis = 0, inplace = True)

games_df = games_df.select_dtypes(exclude=["object"])

games_df = games_df.sort_values("id")

In [4]:
games_df.keys()

Index(['school_id', 'fumblesRecovered', 'rushingTDs', 'passingTDs',
       'firstDowns', 'thirdDownEff', 'totalYards', 'netPassingYards',
       'yardsPerPass', 'rushingYards', 'rushingAttempts',
       'yardsPerRushAttempt', 'turnovers', 'fumblesLost', 'interceptions',
       'poss_total_sec', 'id', 'opposing_netPassingYards',
       'opposing_yardsPerPass', 'opposing_rushingYards',
       'opposing_yardsPerRushAttempt', 'opposing_passingTDs',
       'opposing_rushingTDs', 'thirdDownConverts', 'thirdDownAttempts',
       'fourthDownConverts', 'fourthDownAttempts', 'completions',
       'passAttempts', 'passCompletionPercentage'],
      dtype='object')

We initially set the historical memory equal to 12 games. This is roughly the past seasons-worth of games for an average team. 

In [5]:
memory = 12

In [6]:
teams = set(games_df["school_id"])
print(teams)
print(len(teams))

{2, 2050, 5, 6, 2567, 8, 9, 2569, 2571, 12, 2572, 13, 16, 2065, 2579, 21, 2582, 23, 24, 25, 26, 3101, 30, 2083, 2084, 36, 38, 2598, 41, 43, 47, 48, 2097, 50, 52, 55, 57, 58, 59, 2617, 61, 62, 2110, 2619, 2623, 66, 2115, 2116, 2117, 2628, 68, 70, 2633, 2630, 2635, 2636, 77, 2638, 79, 2127, 2641, 2634, 2643, 84, 2132, 2640, 87, 2649, 2653, 2142, 2655, 96, 97, 98, 99, 93, 103, 107, 113, 2674, 2678, 119, 120, 2681, 2169, 127, 130, 2692, 2181, 135, 2184, 142, 145, 2193, 147, 149, 150, 151, 152, 153, 154, 2711, 2199, 2717, 158, 155, 160, 2210, 164, 166, 167, 2729, 2226, 2229, 2230, 183, 2747, 189, 2751, 193, 194, 195, 2754, 197, 2755, 2247, 2241, 201, 202, 204, 2771, 213, 2261, 218, 221, 222, 227, 228, 2277, 231, 233, 235, 236, 238, 239, 2287, 242, 2803, 245, 2294, 248, 249, 2296, 251, 252, 253, 254, 256, 2305, 258, 259, 2306, 2309, 257, 264, 265, 2320, 275, 276, 277, 278, 2837, 2329, 282, 2335, 290, 2341, 295, 2348, 301, 2197, 302, 304, 2198, 309, 311, 322, 2710, 324, 326, 328, 2377, 331, 3

In [7]:
X = []
y = []
for team in teams:
    team_df = games_df[games_df["school_id"] == team]
    team_df = team_df.sort_values("id")
    team_df.drop(columns=["id", "school_id"], inplace=True)
    team_array = np.array(team_df)
    
    for i in range(memory, len(team_array)):
        X.append(np.hstack(team_array[i-memory:i]))
        y.append(team_array[i])
   

In [8]:
print(len(team_df.keys()))
print(len(y[0]))

28
28


We now split the data into the training and testing sets:

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Now we have the train and test data, we will scale the input to the model. Since the output is stats each working on the same scales as the inputs, we scale the output as well.

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

scaler2 = StandardScaler()
scaler2.fit(y_train)
y_train_scaled = scaler2.transform(y_train)
y_test_scaled = scaler2.transform(y_test)

## Training and testing models
We first try some linear models for predicting the stats in the next game.

In [11]:
from sklearn.linear_model import Ridge, ElasticNet, Lasso, LinearRegression

reg25 = Ridge(alpha=0.25)
reg25.fit(X_train_scaled, y_train_scaled)

reg50 = Ridge(alpha=0.5)
reg50.fit(X_train_scaled, y_train_scaled)

reg100 = Ridge(alpha=1)
reg100.fit(X_train_scaled, y_train_scaled)

Ridge(alpha=1)

We now make predictions with the ridge models on the test set and compute the mean squared error. 

In [12]:
from sklearn.metrics import mean_squared_error

pred_scores25 = reg25.predict(X_test_scaled)
pred_scores50 = reg50.predict(X_test_scaled)
pred_scores100 = reg100.predict(X_test_scaled)

print(f"alpha: MSE are: \n 0.25:{mean_squared_error(pred_scores25, y_test_scaled)} \n 0.50:{mean_squared_error(pred_scores50, y_test_scaled)} \n 1.00:{mean_squared_error(pred_scores100, y_test_scaled)}")

alpha: MSE are: 
 0.25:0.9289504736159343 
 0.50:0.9289502575653117 
 1.00:0.9289266963929623


In [15]:
lasso = Lasso(alpha=1)
lasso.fit(X_train_scaled, y_train_scaled)

ela_net = ElasticNet(alpha=1)
ela_net.fit(X_train_scaled, y_train_scaled)

print(f"MSE are \n Lasso:{mean_squared_error(lasso.predict(X_test_scaled), y_test_scaled)}\n ElasticNet:{mean_squared_error(ela_net.predict(X_test_scaled), y_test_scaled)}")

MSE are 
 Lasso:0.9948268539848032
 ElasticNet:0.9948268539848032


In the previous modelling attempts, we predicted the stats for a given team based only on that team's history. Now we will try to look at predicting stats for a given game based on the combined histories of the two teams playing.

In [102]:
teams_dict = {}
for team in teams:
    temp_df = games_df[games_df["school_id"] == team].sort_values("id")
    #temp_df.reset_index(inplace=True)
    teams_dict[team] = temp_df.reset_index(drop=True)


In [123]:
X2 = []
y2 = []
for team in teams:
    team1_df = teams_dict[team]
    print(f"on team {team}")
    
    # loop over all possible games for a given team with enough prior games
    for i in range(memory, len(team1_df)):
        # find the two teams participating in the game
        game_id = team1_df.loc[i]["id"]
        both_teams_ids = games_df[games_df["id"] == game_id]["school_id"]
        both_teams_ids = list(both_teams_ids)

        # throw out some odd cases where  there was only data for one of the teams in a given game
        if len(both_teams_ids) != 2:
            continue
        #print(f"both teams are {both_teams_ids}")
 
        # extract the dataframes for the two teams, find which game it was for them
        t1_df = teams_dict[both_teams_ids[0]]#.loc[i-memory:i+1]
        t2_df = teams_dict[both_teams_ids[1]]#.loc[i-memory:i+1]

        t1_gamen = np.where(t1_df["id"] == game_id)[0][0]
        t2_gamen = np.where(t2_df["id"] == game_id)[0][0]

        # check to make sure that both teams have enough games in the past
        if t1_gamen < memory or t2_gamen < memory:
            continue
        
        # extract the games in recent history, dropping columns that don't go into the data
        t1_df = t1_df.loc[t1_gamen-memory:t1_gamen]#+1]
        t2_df = t2_df.loc[t2_gamen-memory:t2_gamen]#+1]
        t1_df.drop(columns=["id", "school_id"], inplace=True)
        t2_df.drop(columns=["id", "school_id"], inplace=True)

        t1_array = np.array(t1_df)
        t2_array = np.array(t2_df)

        # break the arrays into past games and current game to go into training and test set
        t1_past = t1_array[0:-1]
        t1_present = t1_array[-1]
        t2_past = t2_array[0:-1]
        t2_present = t2_array[-1]


        X2.append(np.hstack([t1_past, t2_past]))
        y2.append(np.hstack([t1_present, t2_present]))

on team 2
on team 2050
on team 5
on team 6
on team 2567
on team 8
on team 9
on team 2569
on team 2571
on team 12
on team 2572
on team 13
on team 16
on team 2065
on team 2579
on team 21
on team 2582
on team 23
on team 24
on team 25
on team 26
on team 3101
on team 30
on team 2083
on team 2084
on team 36
on team 38
on team 2598
on team 41
on team 43
on team 47
on team 48
on team 2097
on team 50
on team 52
on team 55
on team 57
on team 58
on team 59
on team 2617
on team 61
on team 62
on team 2110
on team 2619
on team 2623
on team 66
on team 2115
on team 2116
on team 2117
on team 2628
on team 68
on team 70
on team 2633
on team 2630
on team 2635
on team 2636
on team 77
on team 2638
on team 79
on team 2127
on team 2641
on team 2634
on team 2643
on team 84
on team 2132
on team 2640
on team 87
on team 2649
on team 2653
on team 2142
on team 2655
on team 96
on team 97
on team 98
on team 99
on team 93
on team 103
on team 107
on team 113
on team 2674
on team 2678
on team 119
on team 120
on team 268

In [131]:
X2_flat = []
for i in range(len(X2)):
    X2_flat.append(X2[i].reshape(-1))

In [132]:
X2_flat = np.array(X2_flat)
y2 = np.array(y2)
print(X2_flat.shape, y2.shape)

(12742, 672) (12742, 56)


Now we scale the data, split it into train and test sets, and then train and test various models.

In [133]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2_flat, y2, test_size=0.25, random_state=42)


scaler = StandardScaler()
scaler.fit(X2_train)
X2_train_scaled = scaler.transform(X2_train)
X2_test_scaled = scaler.transform(X2_test)

scaler2 = StandardScaler()
scaler2.fit(y2_train)
y2_train_scaled = scaler2.transform(y2_train)
y2_test_scaled = scaler2.transform(y2_test)

In [134]:
reg25 = Ridge(alpha=0.25)
reg25.fit(X2_train_scaled, y2_train_scaled)

reg50 = Ridge(alpha=0.5)
reg50.fit(X2_train_scaled, y2_train_scaled)

reg100 = Ridge(alpha=1)
reg100.fit(X2_train_scaled, y2_train_scaled)

Ridge(alpha=1)

In [135]:
pred_scores25 = reg25.predict(X2_test_scaled)
pred_scores50 = reg50.predict(X2_test_scaled)
pred_scores100 = reg100.predict(X2_test_scaled)

print(f"alpha: MSE are: \n 0.25:{mean_squared_error(pred_scores25, y2_test_scaled)} \n 0.50:{mean_squared_error(pred_scores50, y2_test_scaled)} \n 1.00:{mean_squared_error(pred_scores100, y2_test_scaled)}")

alpha: MSE are: 
 0.25:0.8558815223889026 
 0.50:0.8559164936772383 
 1.00:0.855879334615943


We see that the performance is improved by considering the historical data of both teams, not just each team independently.